<a href="https://colab.research.google.com/github/ezhilnn/india_market_snapshot_reporter-notebook/blob/main/india_market_snapshot_reporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================
# 📦 Install Dependencies
# ===========================
!pip install yfinance pandas requests beautifulsoup4 nltk

In [ ]:
# ===========================
# 📚 Import Libraries
# ===========================
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import datetime
import nltk

# Download VADER lexicon for sentiment
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# ===========================
# ⚙️ Configuration
# ===========================

EMAIL_SENDER = "ezhiln03s28@gmail.com"
EMAIL_PASSWORD = "zxlitalbbzfeiklk"   # ⚠️ Use Gmail App Password, not your normal password
EMAIL_RECEIVER = "ezhilneng@gmail.com"

# NSE/BSE sample symbols – expand this list
MARKET_SYMBOLS = [
    "RELIANCE.NS","TCS.NS","HDFCBANK.NS","INFY.NS","ICICIBANK.NS"
]

# RSS feeds for Indian market news
NEWS_SOURCES = [
    "https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms",
    "https://www.business-standard.com/rss/latest.rss",
    "https://www.livemint.com/rss/markets"
]

TOP_N = 50

In [ ]:
# ===========================
# 📊 Stock Data Collection
# ===========================
def get_stock_data(symbols):
    data = []
    for sym in symbols:
        try:
            stock = yf.Ticker(sym)
            hist = stock.history(period="1d")
            if not hist.empty:
                close = hist["Close"].iloc[-1]
                openp = hist["Open"].iloc[-1]
                volume = hist["Volume"].iloc[-1]
                change = ((close - openp) / openp) * 100
                data.append([sym, close, change, volume])
        except Exception as e:
            print(f"⚠️ Error fetching {sym}: {e}")
    df = pd.DataFrame(data, columns=["Symbol","Close","%Change","Volume"])
    gainers = df.sort_values("%Change", ascending=False).head(TOP_N)
    losers = df.sort_values("%Change", ascending=True).head(TOP_N)
    volume = df.sort_values("Volume", ascending=False).head(TOP_N)
    return gainers, losers, volume

In [ ]:
# ===========================
# 📰 News Data Collection
# ===========================
def fetch_news():
    headlines = []
    for url in NEWS_SOURCES:
        try:
            resp = requests.get(url, timeout=10)
            soup = BeautifulSoup(resp.content, "xml")
            items = soup.find_all("item")
            for it in items:
                headlines.append({
                    "title": it.title.text,
                    "link": it.link.text
                })
        except Exception as e:
            print(f"⚠️ Error fetching news: {e}")
    return headlines[:500]

In [ ]:
# ===========================
# 😊 Sentiment & Categorization
# ===========================
def analyze_news(headlines):
    sid = SentimentIntensityAnalyzer()
    positive, negative, business, govt = [], [], [], []
    for h in headlines:
        score = sid.polarity_scores(h["title"])["compound"]
        if score > 0.5:
            positive.append(h)
        elif score < -0.5:
            negative.append(h)
        text = h["title"].lower()
        if any(k in text for k in ["quarterly","merger","acquisition","results","finance","ipo"]):
            business.append(h)
        if any(k in text for k in ["rbi","sebi","policy","tax","budget","ministry","fdi"]):
            govt.append(h)
    return positive[:TOP_N], negative[:TOP_N], business[:TOP_N], govt[:TOP_N]

In [ ]:
# ===========================
# 📝 Report Formatting (HTML)
# ===========================
def format_html(gainers, losers, volume, pos, neg, biz, govt):
    today = datetime.date.today().strftime("%d-%b-%Y")

    def make_list(data):
        if isinstance(data, pd.DataFrame):
            return data.to_html(index=False, border=0)
        else:
            return "<ul>" + "".join([f"<li><a href='{d['link']}'>{d['title']}</a></li>" for d in data]) + "</ul>"

    html = f"""
    <h2>Daily India Market Snapshot – {today}</h2>
    <h3>Top {TOP_N} Gainers</h3>{make_list(gainers)}
    <h3>Top {TOP_N} Losers</h3>{make_list(losers)}
    <h3>Top {TOP_N} Most Traded</h3>{make_list(volume)}
    <h3>Top {TOP_N} Positive News</h3>{make_list(pos)}
    <h3>Top {TOP_N} Negative News</h3>{make_list(neg)}
    <h3>Top {TOP_N} Business News</h3>{make_list(biz)}
    <h3>Top {TOP_N} Government/Regulatory News</h3>{make_list(govt)}
    """
    return html

In [ ]:
# ===========================
# 📧 Send Email
# ===========================
def send_email(subject, body):
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECEIVER
    msg.attach(MIMEText(body, "html"))

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())

In [ ]:
# ===========================
# ▶️ Run the Agent
# ===========================
def main():
    gainers, losers, volume = get_stock_data(MARKET_SYMBOLS)
    headlines = fetch_news()
    pos, neg, biz, govt = analyze_news(headlines)
    html_report = format_html(gainers, losers, volume, pos, neg, biz, govt)
    send_email("Daily India Market Snapshot – " + str(datetime.date.today()), html_report)
    print("✅ Report Sent Successfully!")

main()

✅ Report Sent Successfully!
